In [51]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [52]:
import sys

sys.path.append("..")

In [62]:
import pandas as pd

from src.constants import (
    PROJECT_ROOT_DIR,
    INT_FOLDER,
    ITEM_CLUSTERS_FILE,
    PROCESSED_FOLDER,
    SYNTHETIC_INTERACTIONS_FILE
)

In [54]:
item_cluster_path = PROJECT_ROOT_DIR / INT_FOLDER / ITEM_CLUSTERS_FILE

In [55]:
item_cluster_df = pd.read_csv(item_cluster_path)

item_cluster_df.head()

,item_id,title,cluster
0,5b00c097-464e-5195-9fb4-ce51218ea733,Bulletproof Ruby on Rails Applications,47
1,cbbafad0-63b5-526c-93ff-435ebca16aa9,Business Machine Learning,45
2,1e77eebd-3485-56f6-984e-d0efc70ff119,C++ Brain Teasers: Exercise Your Mind,131
3,5f7953c7-2fe5-5eb3-b39e-444eeafdba97,Calculating Sales Tax Using Avalara's AvaTax A...,84
4,cb662d24-384b-5dd2-b5e6-2a2928c1002d,"Cloud Native Development with Tailwind, Google...",13


In [56]:
len(item_cluster_df)

1119

In [57]:
# Randomly create user

from faker import Faker

fake = Faker()

user_id = fake.bothify(text="?" * 26, letters="ABCDEFGHIJKLMNOPQRSTUVWXYZ")

user_list = list(
    set(
        [
            fake.bothify(text="?" * 26, letters="ABCDEFGHIJKLMNOPQRSTUVWXYZ")
            for _ in range(100)
        ]
    )
)

In [58]:
len(user_list)

100

In [59]:
user_list[:4]

['DSDPCUHFBPJSUQFKKQQNTYVKTA',
 'NZTQPULJLBUPBJGCWGWMOTAICG',
 'MBOIAZWUCZQRMMBJHRLEXRUHCZ',
 'GQVISSXSXJKPXOMYVRYTTKBABP']

'''
loop through cluster group, choose random user in user_list, choose random item in item_list of that group, assign interaction.

loop through item in item_cluster_df, choose random user in user_list, choose 10 items from the group of that item, assign interaction.
'''


In [ ]:
import random

items = item_cluster_df["item_id"]

synthetic_data = []

for item in items:
    item_group = item_cluster_df.loc[item_cluster_df["item_id"] == item][
        "cluster"
    ].values.tolist()[0]
    similar_items = item_cluster_df.loc[item_cluster_df["cluster"].isin([item_group])][
        "item_id"
    ].values.tolist()

    user = random.choice(user_list)

    max_num_items = max(30, len(similar_items))
    for _ in range(max_num_items):
        item = random.choice(similar_items)
        synthetic_data.append((user, item, 1))

synthetic_data_df = pd.DataFrame(
    synthetic_data, columns=["user_id", "item_id", "interaction"]
)

In [61]:
synthetic_data_df

,user_id,item_id,interaction
0,UHJIDWXTNRDWUGFTPLTOENZBLE,75596c12-ae52-5782-a859-97702d2d2b02,1
1,UHJIDWXTNRDWUGFTPLTOENZBLE,0c4b74a6-a40a-5f82-8b84-fe4b315b0ea5,1
2,UHJIDWXTNRDWUGFTPLTOENZBLE,75596c12-ae52-5782-a859-97702d2d2b02,1
3,UHJIDWXTNRDWUGFTPLTOENZBLE,5b00c097-464e-5195-9fb4-ce51218ea733,1
4,UHJIDWXTNRDWUGFTPLTOENZBLE,add18ae7-c406-5cc4-a19f-3ce823242102,1
...,...,...,...
36256,FKKNFYRBMANVNEDXRVKOCDTULL,3f70d070-2ca8-5780-adac-b7633376796d,1
36257,FKKNFYRBMANVNEDXRVKOCDTULL,02a54890-99b8-51e4-9604-285bc64fa3c8,1
36258,FKKNFYRBMANVNEDXRVKOCDTULL,9ca1a2df-6cb9-56e1-9b6e-6445fee8196c,1
36259,FKKNFYRBMANVNEDXRVKOCDTULL,02a54890-99b8-51e4-9604-285bc64fa3c8,1


In [63]:
p = PROJECT_ROOT_DIR / PROCESSED_FOLDER / SYNTHETIC_INTERACTIONS_FILE

synthetic_data_df.to_csv(p, index=False)